In [39]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from polygon import RESTClient
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
import polygon
import os, pickle
from scipy.optimize import minimize
from nelson_siegel_svensson.calibrate import calibrate_nss_ols
from sklearn.preprocessing import MinMaxScaler
from multiprocess import Pool
from joblib import Parallel, delayed
import pickle

In [ ]:
datetime_diff = lambda date1, date2 : (datetime.strptime(date1, '%Y-%m-%d') - datetime.strptime(date2, '%Y-%m-%d')).days

### Data Collection

In [ ]:
load_dotenv("/Users/brad/mlprojects/guidelight/guidelight-api/.env")
polygon_token = os.getenv("POLYGON_TOKEN")

In [ ]:
import pandas as pd

if not os.path.exists("data.pkl"):
    indices = [(contract.ticker, contract.expiration_date, contract.strike_price) for contract in all_contracts]
    data = {}
    for index in indices:
        ticker, expiration_date, strike_price = index
        current_date = datetime.strptime(expiration_date, "%Y-%m-%d")
        past_date = current_date - timedelta(days=14)

        # get key value data for each agg

        a = [vars(agg) for agg in client.get_aggs(ticker, 1, 'day', past_date, current_date)]
        data[index] = a


    pickle.dump(data, open("data.pkl", "wb"))

else:
    data = pickle.load(open("data.pkl", "rb"))

In [ ]:
def get_agg_worker(agg):
    return vars(agg)

def generate_option_aggs(ticker, token=polygon_token):
    client = RESTClient(api_key=token)
    if not os.path.exists(f"cache/{ticker}_contracts.pkl"):
        reqs = client.list_options_contracts(ticker,as_of="2024-04-16", expired=True, expiration_date_gt="2023-04-16")
        all_contracts = list(reqs)
        pickle.dump(all_contracts, open(f"cache/{ticker}_contracts.pkl", "wb"))
    else:
        all_contracts = pickle.load(open(f"cache/{ticker}_contracts.pkl", "rb"))



    indices = [(contract.ticker, contract.expiration_date, contract.strike_price) for contract in all_contracts]
    data = {}
    if not os.path.exists("data.pkl"):
        for index in indices:
            ticker, expiration_date, strike_price = index
            current_date = datetime.strptime(expiration_date, "%Y-%m-%d")
            past_date = current_date - timedelta(days=14)

            # Fetch aggregates for each contract within the date range
            aggs = client.get_aggs(ticker, 1, 'day', past_date, current_date)
            
            # Using Pool for asynchronous map
            with Pool(10) as p:
                async_result = p.map_async(get_agg_worker, aggs)
                p.close()  # No more tasks will be submitted, safe to close the pool
                p.join()  # Wait for all worker processes to finish
                
                # Collect results
                results = async_result.get()
                data[index] = results
    else:
        data = pickle.load(open("data.pkl", "rb"))
    return data

In [ ]:
def save_option_ticker(underlying_ticker:str, data):
# Flatten the data while preserving the option ticker and expiration date
	flattened_data = []
	for (ticker, expiration, strike_price), entries in data.items():
		for entry in entries:
			entry.update({
				"ticker": ticker,
				"expiration_date": expiration,
				"strike_price": strike_price
			})
			flattened_data.append(entry)

	# Create a DataFrame
	df = pd.DataFrame(flattened_data)

	# Set a MultiIndex using the ticker, expiration date, and trading date
	df.set_index(['ticker', "strike_price", 'expiration_date'], inplace=True)

	# get by ticker
	# 1681099200000
	df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
	# df.index = df.index.set_levels(pd.to_datetime(df.index.get_level_values('timestamp'), unit='ms').strftime('%Y-%m-%d %H:%M:%S'), level='timestamp')
	df.to_csv(f"options_contracts/{underlying_ticker.upper()}.csv", index_label=['ticker', "strike_price", 'expiration_date'])

In [ ]:
# Flatten the data while preserving the option ticker and expiration date
flattened_data = []
for (ticker, expiration, strike_price), entries in data.items():
    for entry in entries:
        entry.update({
            "ticker": ticker,
            "expiration_date": expiration,
            "strike_price": strike_price
        })
        flattened_data.append(entry)

# Create a DataFrame
df = pd.DataFrame(flattened_data)

# Set a MultiIndex using the ticker, expiration date, and trading date
df.set_index(['ticker', "strike_price", 'expiration_date'], inplace=True)

# get by ticker
# 1681099200000
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.strftime("%Y-%m-%d")
# df.index = df.index.set_levels(pd.to_datetime(df.index.get_level_values('timestamp'), unit='ms').strftime('%Y-%m-%d %H:%M:%S'), level='timestamp')

In [ ]:
def daily_option_data(underlying_ticker:str, date:str):
	if os.path.exists(f"options_contracts/{underlying_ticker}-{date}.csv"):
		return pd.read_csv(f"options_contracts/{underlying_ticker}-{date}.csv")

	df = pd.read_csv(f'options_contracts/{underlying_ticker}.csv', index_col=[0, 1, 2])
	option_contracts = df.loc[df['timestamp'] == date]
	option_contracts.reset_index(inplace=True)
	# print(option_contracts)
	colnames = ["ticker", "maturity", "Weight", 'price', 'days since last trade', 'strike', 'S']
	volsurface = pd.DataFrame(columns=colnames)

	for ticker in option_contracts['ticker'].unique():
		agg_series= df.loc[(ticker, slice(None), slice(None))]
		i = np.where(agg_series['timestamp'].values == date)[0][0]
	# agg_series
		if i <= 0:
			continue

		diff = datetime_diff(agg_series['timestamp'].iloc[i], agg_series['timestamp'].iloc[i-1])
		if diff <= 3:
			expiration_date = agg_series.index.get_level_values(1).unique()[0]
			time_to_maturity =datetime_diff(expiration_date, date) 
			row = pd.DataFrame({
				'ticker': ticker,
				'maturity': time_to_maturity/365 if time_to_maturity else 6.5/(24 * 365),
				'price': agg_series["vwap"].values[i],
				'Weight': agg_series["volume"].values[i] / agg_series["volume"].sum(),
				'days since last trade': diff,
				'strike': agg_series.index.get_level_values(0).unique()[0],
				'S': agg_series['open'].values[i]
			}, columns=colnames, index=[0])

			volsurface = pd.concat([volsurface, row], ignore_index=True)
			

	
	volsurface.to_csv( os.path.join(os.getcwd(), f"options_data/{underlying_ticker}-{date}.csv"))
	return volsurface

# Multiprocessing 

In [ ]:


def worker(date, underlying_ticker):
    return daily_option_data(underlying_ticker, date)


def process_multiple_days(underlying_ticker, start_date, end_date):
    # Generate list of dates
    dates = mcal.get_calendar("NYSE").valid_days(start_date=start_date, end_date=end_date)
    
    # Define a helper to wrap your existing function for use with starmap
    

    # Setup multiprocessing pool
    # with Pool() as pool:
    #     pool.starmap(worker, [(underlying_ticker, date) for date in dates])

    # print("Data processing complete for all specified dates.")
    dataset = [daily_option_data(underlying_ticker, timestamp.date().strftime("%Y-%m-%d")) for timestamp in dates]

    # return dataset
    return dataset



In [ ]:
# yield_rates = pd.read_csv("five-year-rates.csv")
# yield_maturities = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
# d = datetime.strftime(datetime.strptime("2023-04-18", "%Y-%m-%d"), "%m/%d/%Y")
# yields = yield_rates.loc[yield_rates["Date"]==d].values[:,1:].astype(np.float128).reshape(-1)
# scaler = MinMaxScaler()
# yields_normalized = scaler.fit_transform(yields.reshape(-1, 1)).flatten()
# calibrate_nss_ols(yield_maturities, yields_normalized)

In [ ]:
# hist_voilatilities = torch.empty(len(daily_aggs))
# for i, day in enumerate(daily_aggs):
# 	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# # calculate the historical volatility for each day

In [ ]:

# from heston_param import *
# hist_voilatilities = pickle.load(open("hist_voilatilities.pkl", "rb"))
# daily_aggs = pickle.load(open("daily_aggs.pkl", "rb"))

In [ ]:
# while True:
# 	try:
# 		print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, daily_aggs[0]["close"].values.shape[0], 50))
# 		break
# 	except RuntimeError:
# 		continue

In [ ]:
# def single_day_calibration(args):
# 		i, hist_volatilities, daily_agg = args
# 		while True:
# 			try:
# 				params = calibrate_daily_parameters(hist_volatilities[i], 0.1, day["close"].values, 0.0237, day["close"].values.shape[0], 300)
				
# 				return i, params
# 			except RuntimeError:
# 				continue

In [ ]:
# len(daily_aggs)

### Naive Monte Carlo:
Runtime: 1.15 Hour. Suboptimal.

In [ ]:
# daily_params = np.empty((len(daily_aggs), 5))
# for i, day in enumerate((daily_aggs[:20])):
# 	while True:
# 		try:
# 			daily_params[i] = calibrate_daily_parameters(hist_voilatilities[i], 0.1, daily_aggs[i]["close"].values, 0.0237, daily_aggs[i]["close"].values.shape[0], 50)
# 			if (i + 1) % 10 == 0:
# 					rate =  100 *  np.round((i + 1) /len(daily_params[:20]), 2)
# 					print(f"{rate}% completed.")
			
# 			break
# 		except RuntimeError:
# 			continue



In [ ]:
# def calibrate_worker(args):
# 	single_day_calibration(args)

In [ ]:
# daily_params = pickle.load(open("parameters.pkl", "rb"))

In [ ]:
# runtime is 1.5 hours
# daily_params.shape

In [ ]:
# from multiprocess import Pool

# def calibrate_parameters_multiprocessing(daily_aggs, hist_volatilities):
#     # Prepare arguments for each task
#     tasks = [(i, hist_volatilities[i], daily_aggs[i]) for i in range(len(daily_aggs))]

#     # Number of processes, could be set to the number of CPUs or cores
#     num_processes = 4

#     # Create a multiprocessing pool and map tasks to worker function
#     with Pool(processes=num_processes) as pool:
#         results = pool.map(calibrate_worker, tasks)

#     # Process results
#     daily_params = np.empty((len(daily_aggs[:20]), 5))
#     for index, params in results:
#         if params is not None:
#             daily_params[index] = params
#         else:
#             print(f"Calibration failed for index {index}")

#     return daily_params



In [ ]:
# calibrate_parameters_multiprocessing(daily_aggs[:20], hist_voilatilities[:20])

###  Simulation using FFT for the characteristic equations

https://medium.com/@alexander.tsoskounoglou/pricing-options-with-fourier-series-p3-the-heston-model-d157369a217a

In [ ]:
def heston_char(u, params):
    kappa, theta, zeta, rho, v0, r, q, T, S0 = params 
    t0 = 0.0 ;  q = 0.0
    m = np.log(S0) + (r - q)*(T-t0)
    D = np.sqrt((rho*zeta*1j*u - kappa)**2 + zeta**2*(1j*u + u**2))
    C = (kappa - rho*zeta*1j*u - D) / (kappa - rho*zeta*1j*u + D)
    beta = ((kappa - rho*zeta*1j*u - D)*(1-np.exp(-D*(T-t0)))) / (zeta**2*(1-C*np.exp(-D*(T-t0))))
    alpha = ((kappa*theta)/(zeta**2))*((kappa - rho*zeta*1j*u - D)*(T-t0) - 2*np.log((1-C*np.exp(-D*(T-t0))/(1-C))))
    return np.exp(1j*u*m + alpha + beta*v0)

In [ ]:
import numpy as np
from numpy import sqrt, exp, pi, cos, sin, log, abs
from numba import njit

@njit(parallel=True)
def Fourier_Heston_Put(S0, K, T, r, 
                  # Heston Model Paramters
                  kappa, # Speed of the mean reversion 
                  theta, # Long term mean
                  rho,   # correlation between 2 random variables
                  zeta,  # Volatility of volatility
                  v0,    # Initial volatility 
                  opt_type,
                  N = 1_012,
                  z = 24
                  ):

  def heston_char(u): 
    t0 = 0.0 ;  q = 0.0
    m = log(S0) + (r - q)*(T-t0)
    D = sqrt((rho*zeta*1j*u - kappa)**2 + zeta**2*(1j*u + u**2))
    C = (kappa - rho*zeta*1j*u - D) / (kappa - rho*zeta*1j*u + D)
    beta = ((kappa - rho*zeta*1j*u - D)*(1-exp(-D*(T-t0)))) / (zeta**2*(1-C*exp(-D*(T-t0))))
    alpha = ((kappa*theta)/(zeta**2))*((kappa - rho*zeta*1j*u - D)*(T-t0) - 2*log((1-C*exp(-D*(T-t0))/(1-C))))
    return exp(1j*u*m + alpha + beta*v0)
  
  # # Parameters for the Function to make sure the approximations are correct.
  c1 = log(S0) + r*T - .5*theta*T
  c2 = theta/(8*kappa**3)*(-zeta**2*exp(-2*kappa*T) + 4*zeta*exp(-kappa*T)*(zeta-2*kappa*rho) 
        + 2*kappa*T*(4*kappa**2 + zeta**2 - 4*kappa*zeta*rho) + zeta*(8*kappa*rho - 3*zeta))
  a = c1 - z*sqrt(abs(c2))
  b = c1 + z*sqrt(abs(c2))
  
  h       = lambda n : (n*pi) / (b-a) 
  g_n     = lambda n : (exp(a) - (K/h(n))*sin(h(n)*(a - log(K))) - K*cos(h(n)*(a - log(K)))) / (1 + h(n)**2)
  g0      = K*(log(K) - a - 1) + exp(a)
  
  F = g0 
  for n in range(1, N+1):
    h_n = h(n)
    F += 2*heston_char(h_n) * exp(-1j*a*h_n) * g_n(n)

  F = exp(-r*T)/(b-a) * np.real(F)
  F = F if opt_type == 'p' else F + S0 - K*exp(-r*T)
  return F if F > 0 else 0




In [ ]:
S0      = 100.      # initial asset price
K       = 50.       # strike
r       = 0.03      # risk free rate
T       = 1/365     # time to maturity

v0=0.4173 ; kappa=0.4352 ; theta=0.2982 ; zeta=1.3856 ; rho=-0.0304

In [ ]:
import py_vollib_vectorized

price = 0.10 ; S = 95 ; K = 100 ; t = .2 ; r = .2 ; flag = 'c'

def implied_volatility(price, S, K, t, r, flag):
  return py_vollib_vectorized.vectorized_implied_volatility(
    price, S, K, t, r, flag, q=0.0, on_error='ignore', model='black_scholes_merton',return_as='numpy') 


In [ ]:
# %pip install pyFFTW
import pyfftw

In [ ]:
import numpy as np
from numpy import exp, pi, log, sqrt
# from  numpy.fft import fft
import pyfftw
@njit(parallel=True)
def heston_fft2(S0, K, T, r, kappa, theta, rho, zeta, v0, opt_type, N=1024, alpha=1.5):
    eta = 0.25  # Grid spacing for the integration variable
    lambda_u = 2 * pi / (N * eta)  # Grid spacing for log strike

    # Adjustments for the damping factor
    alpha = alpha  # Damping factor, typically 1 or 1.5

    # Characteristic function for the Heston model as before
    def heston_char(u):
        t0 = 0.0 ; q = 0.0
        m = log(S0) + (r - q) * (T - t0)
        D = sqrt((rho * zeta * 1j * u - kappa) ** 2 + zeta ** 2 * (1j * u + u ** 2))
        C = (kappa - rho * zeta * 1j * u - D) / (kappa - rho * zeta * 1j * u + D)
        beta = ((kappa - rho * zeta * 1j * u - D) * (1 - exp(-D * (T - t0)))) / (zeta ** 2 * (1 - C * exp(-D * (T - t0))))
        alpha = ((kappa * theta) / (zeta ** 2)) * ((kappa - rho * zeta * 1j * u - D) * (T - t0) - 2 * log((1 - C * exp(-D * (T - t0))) / (1 - C)))
        return exp(1j * u * m + alpha + beta * v0)

    # Array of discretized u values (integration variable)
    u = np.arange(1, N) * eta
    u = np.hstack((np.array([0.000001]), u))  # Avoid division by zero in calculations

    # Weights for the integration
    weights = np.ones(N)
    weights[0] = 0.5  # Trapezoidal rule: first weight is 0.5
    weights = weights * eta

    # Damping factor applied to characteristic function
    adjusted_char_fn = exp(-r * T) * (heston_char(u - (alpha + 1) * 1j) / (alpha ** 2 + alpha - u ** 2 + 1j * (2 * alpha + 1) * u))

    # FFT calculation
    fft_object = pyfftw.builders.rfft(adjusted_char_fn.real * weights, threads=4)
    fft_values = fft_object()
    fft_values = fft_values[:N // 2]  # Only need the first half of the FFT output

    # Calculate strike prices corresponding to FFT output
    strikes = S0 * exp(-lambda_u * np.arange(N // 2))

    # Option prices
    prices = np.exp(-alpha * np.log(strikes)) / pi * fft_values.real

    # Find the index of the strike closest to K
    index = np.argmin(np.abs(strikes - K))
    price = prices[index]
    return price if opt_type == 'p' else price + S0 - K * exp(-r * T)


                

In [ ]:
# from nelson_siegel_svensson.calibrate import calibrate_nss_ols
# yield_maturities = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
# # yields  = np.array([5.30,5.39,5.50,5.50,5.44,5.11,4.33,3.98,3.70,3.66,3.61,3.98,3.84])
# # get the first row of the yield rates
# yields = yield_rates.iloc[0].values[1:].astype(np.float64)
# curve_fit, status = calibrate_nss_ols(yield_maturities,yields)
# # yields

In [ ]:
# curve_fit, status

In [ ]:
def get_implied_volatility(price, S, K, t, r, flag):
    return py_vollib_vectorized.vectorized_implied_volatility(
        price, S, K, t, r, flag, q=0.0, on_error='ignore', model='black_scholes_merton',return_as='numpy') 

In [ ]:
def print_debug_info(v0, kappa, theta, zeta, rho, wmae, idx, zeros, rmse):
    print(
        f">>v0={v0:.4f}; kappa={kappa:.4f}; theta={theta:.4f}; "
        f"zeta={zeta:.4f}; rho={rho:7.4f} | WMAE(IV): {wmae:.5e} | "
        f"Nulls: {idx.sum()}/{idx.shape[0]} | Zeros: {zeros}/{idx.shape[0]} | "
        f"WRMSE(IV): {rmse:.5e}"
    )
def SqErr(x, volSurface, _S, _K, _T, _r, _IV, _Weight):
    
    v0, kappa, theta, zeta, rho = x

    # Calculate prices using Heston Model
    Price_Heston = get_resutls_array_Heston(
        volSurface, v0, kappa, theta, zeta, rho, N=1_012, z=24
    )
    
    # Calculate implied volatilities
    IV_Heston = get_implied_volatility(
        price=Price_Heston, S=_S, K=_K, t=_T, r=_r, flag='p'
    )
    
    # Handle undefined IV calculations
    diff = IV_Heston - _IV
    idx = np.isnan(diff) | np.isinf(diff)
    diff[idx] = 0 - _IV[idx]
    IV_Heston[idx] = 0
    diff = np.nan_to_num(diff, 0)
    # Calculate RMSE
    rmse = sqrt(np.mean((diff * 100) ** 2 * _Weight))
    
    # Debugging info
    zeros = int(np.where(IV_Heston == 0, 1, 0).sum())
    wmae  = np.mean(np.abs(diff * 100) * _Weight)
    print_debug_info(v0, kappa, theta, zeta, rho, wmae, idx, zeros, rmse)
    return rmse
def get_error_Heston(volSurface, v0, kappa, theta, zeta, rho):
    """Calculates the error between the Heston model and the market prices.
    Arguments:
        volSurface: DataFrame with the market prices.
        v0: Initial variance.
        kappa: Mean reversion speed.
        theta: Long-run variance.
        zeta: Volatility of volatility.
        rho: Correlation between the variance and the asset.
    """
    error = 0
    for _, row in volSurface.iterrows():
        P = row['price']
        HP = Fourier_Heston_Put(S0=row['S'], K=row['strike'], v0=v0, kappa=kappa, theta=theta, zeta=zeta, rho=rho, T=row['maturity'], r=row['rate'], N=2048)
        error += (P - HP)**2

    return error / volSurface.shape[0]

def get_resutls_array_Heston(volSurface, v0, kappa, theta, zeta, rho, N=10_000, z=64):
    # Initialize the results array
    results = -np.ones(volSurface.shape[0])
    # reset the index of the options dataframe
    volSurface.index = np.arange(0, volSurface.shape[0])
    # loop through the rows of the options dataframe and run the Fourier_Heston_Put function
   
    for idx, row in volSurface.iterrows():
        results[idx] = Fourier_Heston_Put(S0=int(row['S']), K=int(row['strike']), v0=v0, kappa=kappa, theta=theta, zeta=zeta, rho=rho, T=row['maturity'], r=row['rate'], N=N, opt_type='p',z=z)
    return results

def get_resutls_df_Heston(volSurface, v0, kappa, theta, zeta, rho, N=2048, z=100):
    observed = volSurface.copy(deep=True)
    heston = volSurface.copy(deep=True)
    observed['source'] = 'Observed'
    heston['source'] = 'Heston Model'

    heston_prices = [] 
    implied_volatilities = []
    for _, row in volSurface.iterrows():
        heston_price = Fourier_Heston_Put(S0=row['S'], K=row['strike'], v0=v0, kappa=kappa, theta=theta, zeta=zeta, rho=rho, T=row['maturity'], r=row['rate'], N=N, opt_type='p', z=z)
        heston_prices.append(heston_price)
        # np.array(... , ndmin=1) So the type of the input is compatible with what numba expects
        maturity  = np.array(row['maturity'],ndmin=1)
        observed_price  = np.array(heston_price,ndmin=1)
        S0 = np.array(row['S'],ndmin=1)
        K  = np.array(row['strike'],ndmin=1)
        r  = np.array(row['rate'],ndmin=1)
        implied_volatility = get_implied_volatility(price=observed_price, S=S0, K=K, t=maturity, r=r, flag=option_type)
        implied_volatilities.append(implied_volatility[0])

    heston['price'] = heston_prices
    heston['IV']    = implied_volatilities

    return pd.concat([observed, heston])

def get_error_df_Heston(volSurface, v0, kappa, theta, zeta, rho, diff='Price', error='Error', weighted=True, N=10_000, z=64):
    if   error == 'Error':          _name = f'Weighted Error {diff}'             if weighted else f'Error {diff}'
    elif error == 'Perc Error':     _name = f'Weighted Persentage Error {diff}'  if weighted else f'Persentage Error {diff}'
    elif error == 'Squared Error':  _name = f'Weighted Squared Error {diff}'     if weighted else f'Squared Error {diff}'
    else: raise Exception("Error: variable 'error' is not defined correctly")
    
    results_df = {'strike':[], 'maturity':[], _name:[], 'Opt. Type':[], 'Weight':[]}

    for _, row in volSurface.copy(deep=True).iterrows():
        _P = Fourier_Heston_Put(S0=row['S'], K=row['strike'], v0=v0, kappa=kappa, theta=theta, zeta=zeta, rho=rho, T=row['maturity'], r=row['rate'], N=N, z=z, opt_type=row['Type'])
        # np.array(... , ndmin=1) So the type of the input is compatible with what numba expects
        _T  = np.array(row['maturity'],ndmin=1)
        _C  = np.array(_P,ndmin=1)
        _P  = np.array(row['price'],ndmin=1)
        _S0 = np.array(row['S'],ndmin=1)
        _K  = np.array(row['strike'],ndmin=1)
        _r  = np.array(row['rate'],ndmin=1)

        _IV  = get_implied_volatility(price=_C, S=_S0, K=_K, t=_T, r=_r, flag='p')
        _IV2 = get_implied_volatility(price=row['price'], S=_S0, K=_K, t=_T, r=_r, flag='p')

        if error    == 'Error':
            if diff == 'IV':  _error  = (_IV - _IV2) *                (row['Weight'] if weighted else 1)
            else           :  _error  = (_C - _P) *                   (row['Weight'] if weighted else 1)
        elif error  == 'Perc Error':
            if diff == 'IV':  _error  = ((_IV - _IV2)/_IV2) * 100 *   (row['Weight'] if weighted else 1)
            else           :  _error  = ((_C - _P)/_P) * 100 *        (row['Weight'] if weighted else 1)
        elif error  == 'Squared Error':
            if diff == 'IV':  _error  = (_IV - _IV2)**2 *             (row['Weight'] if weighted else 1)
            else           :  _error  = (_C - _P)**2 *                (row['Weight'] if weighted else 1)

        results_df[_name].append(_error[0])
        results_df['maturity'].append(_T[0])
        results_df['strike'].append(_K[0])
        results_df['Weight'].append(row['Weight']*10)

    return pd.DataFrame(results_df)

In [ ]:
import py_vollib_vectorized
def heston_volSurface(cleaned_df, yields):
 
    volSurface = cleaned_df.drop(columns=['days since last trade', 'ticker'])
   

    

    def implied_volatility(price, S, K, t, r, flag):

        return py_vollib_vectorized.vectorized_implied_volatility(
            price, S, K, t, r, flag, q=0.0, on_error='ignore', model='black_scholes_merton',return_as='numpy')

    yield_maturities = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
    # yields  = np.array([5.30,5.39,5.50,5.50,5.44,5.11,4.33,3.98,3.70,3.66,3.61,3.98,3.84])
    # get the first row of the yield rates
    yield_rates = pd.read_csv("five-year-rates.csv")
    d = datetime.strftime(datetime.strptime("2023-04-18", "%Y-%m-%d"), "%m/%d/%Y")
    yields = yield_rates.loc[yield_rates["Date"]==d].values[:,1:].astype(np.float64).reshape(-1)
    scaler = MinMaxScaler()
    yields_normalized = scaler.fit_transform(yields.reshape(-1, 1)).flatten()
    curve_fit, _ = calibrate_nss_ols(yield_maturities, yields_normalized)
    volSurface['rate'] = volSurface['maturity'].apply(curve_fit) / 100
    volSurface['IV'] = implied_volatility(volSurface['price'], volSurface['S'], volSurface['strike'], volSurface['maturity'], volSurface['rate'], 'p')
    return volSurface

def heston_daily_volSurface(underlying_ticker, date):
    cleaned = daily_option_data(underlying_ticker, date)
    yield_rates = pd.read_csv("five-year-rates.csv")
   
    d = datetime.strftime(datetime.strptime(date, "%Y-%m-%d"), "%m/%d/%Y")
    yields = yield_rates.loc[yield_rates["Date"]==d].values[:,1:].astype(np.float64).reshape(-1)
    
    volSurface = heston_volSurface(cleaned, yields)
    return volSurface


def heston_parameters(VolSurface):
	# Extract data from dailyVolSurface DataFrame
    _K = VolSurface['strike'].to_numpy()

    _C = VolSurface['price'].to_numpy()
    _T      = VolSurface['maturity'].to_numpy()
    _r      = VolSurface['rate'].to_numpy()
    _S      = VolSurface['S'].to_numpy()
    _IV     = VolSurface['IV'].to_numpy()
    _Weight = VolSurface['Weight'].to_numpy()
    # Initial parameters and bounds for optimization
    params = {
        "v0": {"x0": np.random.uniform(1e-3, 1.2), "lbub": [1e-3, 1.2]},
        "kappa": {"x0": np.random.uniform(1e-3, 10), "lbub": [1e-3, 10]},
        "theta": {"x0": np.random.uniform(1e-3, 1), "lbub": [1e-3, 1.2]},
        "zeta": {"x0": np.random.uniform(1e-2, 4), "lbub": [1e-2, 4]},
        "rho": {"x0": np.random.uniform(-1, 1), "lbub": [-1, 1]}
    }
    x0 = [param["x0"] for _, param in params.items()]
    bnds = [param["lbub"] for _, param in params.items()]
    result = minimize(
    SqErr, x0, args=(VolSurface,  _S, _K, _T, _r, _IV, _Weight),  tol=1e-5, method='SLSQP',
    options={'maxiter': 80, 'ftol': 1e-5, 'disp': True},
    bounds=bnds, jac='3-point'
	)

    return result.x

def heston_day_params(underlying_ticker, date):
    volSurface = heston_daily_volSurface(underlying_ticker, date)
    return heston_parameters(volSurface)

In [ ]:
from numpy.linalg import LinAlgError
def heston_params(underlying_ticker, start_date, end_date):
	dates = mcal.get_calendar("NYSE").valid_days(start_date=start_date, end_date=end_date)
	# params = [heston_day_params(underlying_ticker, date.date().strftime("%Y-%m-%d")) for date in dates]
	df = pd.DataFrame(columns=["date", 'v0', 'kappa', 'theta', 'zeta', 'rho'])
	for date in dates:
		print("optimizing for", date.date().strftime("%Y-%m-%d"), f"for {underlying_ticker}")
		tries = 0
		while tries < 3:
			try:

				params = heston_day_params(underlying_ticker, date.date().strftime("%Y-%m-%d"))
				row = pd.DataFrame({
					"date": date.date().strftime("%Y-%m-%d"),
					'v0': params[0],
					'kappa': params[1],
					'theta': params[2],
					'zeta': params[3],
					'rho': params[4]
				}, columns=["date", 'v0', 'kappa', 'theta', 'zeta', 'rho'], index=[0])
				df = pd.concat([df, row])
				break
			except LinAlgError:
				print("LinAlgError, trying again")
				tries += 1
				continue

	return df

In [ ]:
# underlying_ticker = "jnj"
# underlying_ticker = underlying_ticker.upper()

In [ ]:
# data = generate_option_aggs(underlying_ticker)
# save_option_ticker(underlying_ticker, data)
# dfs = process_multiple_days(underlying_ticker, '2023-04-11', '2024-04-12')

In [42]:
def worker(underlying_ticker):
	# load_dotenv("/Users/brad/mlprojects/guidelight/guidelight-api/.env")
	data = generate_option_aggs(underlying_ticker)
	save_option_ticker(underlying_ticker, data)
	dfs = process_multiple_days(underlying_ticker, '2023-04-11', '2024-04-12')
def heston_worker(underlying_ticker):
	params = heston_params(underlying_ticker, '2023-04-11', '2024-04-12')
	params.to_csv(f"heston_params/{underlying_ticker}.csv")

In [47]:

if __name__ == "__main__":
	tickers = pd.read_csv("sp_tickers.csv")
	ticker_l = tickers["Symbol"].values[:8]
	
	# underlying_ticker = "jnj"
	# underlying_ticker = underlying_ticker.upper()
	Parallel(n_jobs=4)(delayed(worker)(ticker) for ticker in ticker_l)
	Parallel(n_jobs=4)(heston_worker(ticker) for ticker in ticker_l)
	
	# print(params_data)

/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_61113/4284329225.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_61113/4284329225.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_61113/4284329225.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA

optimizing for 2023-04-11 for MMM


/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_61113/4284329225.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  volsurface = pd.concat([volsurface, row], ignore_index=True)
/Users/brad/.local/share/virtualenvs/guidelight-Popvxsqg/lib/python3.11/site-packages/numba/core/typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_61113/3429261243.py", line 5:
<source missing, REPL/exec in use?>

  warnings.warn(erro

>>v0=0.2715; kappa=4.3918; theta=0.3616; zeta=0.6845; rho= 0.2092 | WMAE(IV): 1.87044e+01 | Nulls: 3/57 | Zeros: 32/57 | WRMSE(IV): 1.56842e+02
>>v0=0.2715; kappa=4.3918; theta=0.3616; zeta=0.6845; rho= 0.2092 | WMAE(IV): 1.87044e+01 | Nulls: 3/57 | Zeros: 32/57 | WRMSE(IV): 1.56841e+02
>>v0=0.2715; kappa=4.3918; theta=0.3616; zeta=0.6845; rho= 0.2092 | WMAE(IV): 1.87044e+01 | Nulls: 3/57 | Zeros: 32/57 | WRMSE(IV): 1.56842e+02
>>v0=0.2715; kappa=4.3918; theta=0.3616; zeta=0.6845; rho= 0.2092 | WMAE(IV): 1.87044e+01 | Nulls: 3/57 | Zeros: 32/57 | WRMSE(IV): 1.56842e+02
>>v0=0.2715; kappa=4.3918; theta=0.3616; zeta=0.6845; rho= 0.2092 | WMAE(IV): 1.87043e+01 | Nulls: 3/57 | Zeros: 32/57 | WRMSE(IV): 1.56841e+02
>>v0=0.2715; kappa=4.3918; theta=0.3616; zeta=0.6845; rho= 0.2092 | WMAE(IV): 1.87053e+01 | Nulls: 3/57 | Zeros: 32/57 | WRMSE(IV): 1.56857e+02
>>v0=0.2715; kappa=4.3918; theta=0.3616; zeta=0.6845; rho= 0.2092 | WMAE(IV): 1.87035e+01 | Nulls: 3/57 | Zeros: 32/57 | WRMSE(IV): 1.56

/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_61113/4059996333.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row])
/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_61113/4284329225.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  volsurface = pd.concat([volsurface, row], ignore_index=True)


...

/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_61113/4284329225.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  volsurface = pd.concat([volsurface, row], ignore_index=True)


>>v0=0.2030; kappa=9.8705; theta=0.4417; zeta=3.2043; rho=-0.7921 | WMAE(IV): 2.49822e+01 | Nulls: 15/125 | Zeros: 90/125 | WRMSE(IV): 1.61492e+02
>>v0=0.2030; kappa=9.8705; theta=0.4417; zeta=3.2043; rho=-0.7921 | WMAE(IV): 2.49822e+01 | Nulls: 15/125 | Zeros: 90/125 | WRMSE(IV): 1.61492e+02
>>v0=0.2030; kappa=9.8705; theta=0.4417; zeta=3.2043; rho=-0.7921 | WMAE(IV): 2.49822e+01 | Nulls: 15/125 | Zeros: 90/125 | WRMSE(IV): 1.61492e+02
>>v0=0.2030; kappa=9.8705; theta=0.4417; zeta=3.2043; rho=-0.7921 | WMAE(IV): 2.49822e+01 | Nulls: 15/125 | Zeros: 90/125 | WRMSE(IV): 1.61492e+02
>>v0=0.2030; kappa=9.8706; theta=0.4417; zeta=3.2043; rho=-0.7921 | WMAE(IV): 2.49822e+01 | Nulls: 15/125 | Zeros: 90/125 | WRMSE(IV): 1.61492e+02
>>v0=0.2030; kappa=9.8705; theta=0.4417; zeta=3.2043; rho=-0.7921 | WMAE(IV): 2.49823e+01 | Nulls: 15/125 | Zeros: 90/125 | WRMSE(IV): 1.61493e+02
>>v0=0.2030; kappa=9.8705; theta=0.4417; zeta=3.2043; rho=-0.7921 | WMAE(IV): 2.49821e+01 | Nulls: 15/125 | Zeros: 90/

/Users/brad/.local/share/virtualenvs/guidelight-Popvxsqg/lib/python3.11/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


>>v0=0.2605; kappa=4.7890; theta=0.5258; zeta=3.1366; rho=-1.0000 | WMAE(IV): 2.03547e+01 | Nulls: 11/125 | Zeros: 98/125 | WRMSE(IV): 1.53796e+02
>>v0=0.2605; kappa=4.7890; theta=0.5258; zeta=3.1366; rho=-1.0000 | WMAE(IV): 2.03547e+01 | Nulls: 11/125 | Zeros: 98/125 | WRMSE(IV): 1.53796e+02
>>v0=1.2000; kappa=7.0591; theta=0.5866; zeta=0.0100; rho=-0.9896 | WMAE(IV): 2.87103e+01 | Nulls: 11/125 | Zeros: 84/125 | WRMSE(IV): 1.81846e+02
>>v0=0.3544; kappa=5.0160; theta=0.5319; zeta=2.8239; rho=-0.9989 | WMAE(IV): 2.15980e+01 | Nulls: 11/125 | Zeros: 97/125 | WRMSE(IV): 1.56329e+02
>>v0=0.2699; kappa=4.8117; theta=0.5264; zeta=3.1053; rho=-0.9999 | WMAE(IV): 2.12591e+01 | Nulls: 11/125 | Zeros: 97/125 | WRMSE(IV): 1.55353e+02
>>v0=0.2614; kappa=4.7913; theta=0.5259; zeta=3.1334; rho=-1.0000 | WMAE(IV): 2.03556e+01 | Nulls: 11/125 | Zeros: 98/125 | WRMSE(IV): 1.53794e+02
>>v0=0.2614; kappa=4.7913; theta=0.5259; zeta=3.1334; rho=-1.0000 | WMAE(IV): 2.03556e+01 | Nulls: 11/125 | Zeros: 98/

KeyboardInterrupt: 